In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
# ---- LLM ----
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

In [3]:
# ---- State ----
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [5]:
# ---- Nodes ----
def create_outline(state: BlogState) -> BlogState:
    prompt = f"Generate a detailed blog outline on the topic: {state['title']}"
    response = model.invoke(prompt)
    state["outline"] = response.content
    return state

def create_blog(state: BlogState) -> BlogState:
    prompt = f"Write a detailed blog on '{state['title']}' using the outline:\n{state['outline']}"
    response = model.invoke(prompt)
    state["content"] = response.content
    return state

In [6]:
# ---- Graph ----
graph = StateGraph(BlogState)
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)

graph.set_entry_point("create_outline")
graph.add_edge("create_outline", "create_blog")
graph.set_finish_point("create_blog")

workflow = graph.compile()

In [7]:
# ---- Run ----
result = workflow.invoke({"title": "Rise of AI in India"})
print(result)

{'title': 'Rise of AI in India', 'outline': "## Blog Outline: The Rise of AI in India - A Technological and Societal Transformation\n\n**I. Introduction (approx. 150 words)**\n\n* **Hook:** Start with a compelling statistic or anecdote showcasing AI's growing presence in India (e.g., investment figures, a specific AI application impacting daily life).\n* **Brief overview:** Define AI and its various applications.\n* **Thesis statement:**  Highlight the rapid growth of AI in India, exploring its drivers, challenges, and potential impact on the nation's economy and society.\n* **Roadmap:** Briefly outline the sections to be covered in the blog.\n\n\n**II. Drivers of AI Growth in India (approx. 300 words)**\n\n* **Government Initiatives:**\n    *  Discuss policies like the National AI Strategy, Digital India, and their role in fostering AI development.\n    *  Mention funding and support programs for AI research and startups.\n* **Technological Talent Pool:**\n    *  Highlight India's lar